In [1]:
import sqlite3
import pandas as pd


In [2]:

# Connect to the database file you created
conn = sqlite3.connect("global_energy.db")

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]
print("Tables in the database:", tables)

Tables in the database: ['power_plants', 'world_bank', 'country_capacity', 'iea_balances', 'oecd_greengrowth']


In [4]:
for table in tables:
    df_preview = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5;", conn)
    print(f"\n=== Preview of {table} ===")
    print(df_preview)
    print(df_preview.columns)


=== Preview of power_plants ===
  country country_long                                              name  \
0     AFG  Afghanistan      Kajaki Hydroelectric Power Plant Afghanistan   
1     AFG  Afghanistan                                      Kandahar DOG   
2     AFG  Afghanistan                                      Kandahar JOL   
3     AFG  Afghanistan     Mahipar Hydroelectric Power Plant Afghanistan   
4     AFG  Afghanistan  Naghlu Dam Hydroelectric Power Plant Afghanistan   

   latitude  longitude primary_fuel  capacity_mw commissioning_year  \
0    32.322    65.1190        Hydro         33.0               None   
1    31.670    65.7950        Solar         10.0               None   
2    31.623    65.7920        Solar         10.0               None   
3    34.556    69.4787        Hydro         66.0               None   
4    34.641    69.7170        Hydro        100.0               None   

  generation_gwh_2013 generation_gwh_2014 generation_gwh_2015  \
0                N

In [5]:
for table in tables:
    row_count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table};", conn)["count"][0]
    df_sample = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 1;", conn)
    col_count = len(df_sample.columns)
    print(f"Table '{table}' has {row_count} rows and {col_count} columns.")

Table 'power_plants' has 34936 rows and 13 columns.
Table 'world_bank' has 37511 rows and 67 columns.
Table 'country_capacity' has 167 rows and 3 columns.
Table 'iea_balances' has 6832 rows and 59 columns.
Table 'oecd_greengrowth' has 677449 rows and 6 columns.


In [6]:
# Example: Summarize capacity by primary_fuel
query_fuel_capacity = """
SELECT primary_fuel, COUNT(*) as plant_count, SUM(capacity_mw) as total_capacity
FROM power_plants
GROUP BY primary_fuel
ORDER BY total_capacity DESC
;
"""
df_fuel_capacity = pd.read_sql_query(query_fuel_capacity, conn)
print(df_fuel_capacity)

      primary_fuel  plant_count  total_capacity
0             Coal         2330    1.965541e+06
1              Gas         3998    1.493051e+06
2            Hydro         7156    1.053160e+06
3          Nuclear          195    4.079118e+05
4             Wind         5344    2.630537e+05
5              Oil         2320    2.618787e+05
6            Solar        10665    1.883123e+05
7          Biomass         1430    3.428130e+04
8            Waste         1068    1.474871e+04
9       Geothermal          189    1.268775e+04
10    Cogeneration           41    4.048000e+03
11           Other           43    3.612860e+03
12         Petcoke           12    2.424577e+03
13         Storage          135    1.712300e+03
14  Wave and Tidal           10    5.522000e+02


In [7]:
query_missing = """
SELECT COUNT(*) as missing_capacity
FROM power_plants
WHERE capacity_mw IS NULL OR capacity_mw = 0
"""
df_missing = pd.read_sql_query(query_missing, conn)
print(df_missing)

   missing_capacity
0                 0


In [8]:
query_countries = """
SELECT country, COUNT(*) AS plant_count
FROM power_plants
GROUP BY country
ORDER BY plant_count DESC
LIMIT 10;
"""
df_top_countries = pd.read_sql_query(query_countries, conn)
print("Countries with the most power plants (top 10):")
print(df_top_countries)

Countries with the most power plants (top 10):
  country  plant_count
0     USA         9833
1     CHN         4235
2     GBR         2751
3     BRA         2360
4     FRA         2155
5     IND         1589
6     DEU         1309
7     CAN         1159
8     ESP          829
9     RUS          545


In [9]:
df_wb_indicators = pd.read_sql_query("""
SELECT DISTINCT [Series Name] as indicator
FROM world_bank
ORDER BY indicator
""", conn)
print("Unique World Bank indicators:", df_wb_indicators.shape[0])
print(df_wb_indicators.head(20))

Unique World Bank indicators: 142
                                            indicator
0                                                None
1   Adjusted net savings, excluding particulate em...
2   Adjusted net savings, excluding particulate em...
3   Adjusted net savings, including particulate em...
4   Adjusted net savings, including particulate em...
5       Adjusted savings: energy depletion (% of GNI)
6    Adjusted savings: energy depletion (current US$)
7   Adjusted savings: particulate emission damage ...
8   Adjusted savings: particulate emission damage ...
9   Agriculture, forestry, and fishing, value adde...
10  Alternative and nuclear energy (% of total ene...
11                             Broad money (% of GDP)
12  Carbon dioxide (CO2) emissions (total) excludi...
13  Carbon dioxide (CO2) emissions (total) excludi...
14  Carbon dioxide (CO2) emissions excluding LULUC...
15  Carbon dioxide (CO2) emissions from Agricultur...
16  Carbon dioxide (CO2) emissions from Building

In [10]:
pd.read_sql_query("PRAGMA table_info(power_plants);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,country,TEXT,0,None,0
1,1,country_long,TEXT,0,None,0
2,2,name,TEXT,0,None,0
3,3,latitude,REAL,0,None,0
4,4,longitude,REAL,0,None,0
5,5,primary_fuel,TEXT,0,None,0
6,6,capacity_mw,REAL,0,None,0
7,7,commissioning_year,REAL,0,None,0
8,8,generation_gwh_2013,REAL,0,None,0
9,9,generation_gwh_2014,REAL,0,None,0


In [11]:
# Example: Join power_plants with a few columns from world_bank on iso3
query_join = """
SELECT p.country, p.primary_fuel, p.capacity_mw,
       w."Series Name" AS wb_indicator,
       w."2020 [YR2020]" AS wb_2020_value
FROM power_plants p
LEFT JOIN world_bank w
    ON p.country = w.country
WHERE w."Series Name" = 'Carbon dioxide (CO2) emissions (kt)'
LIMIT 50;
"""
df_join_example = pd.read_sql_query(query_join, conn)
print(df_join_example.head(10))

Empty DataFrame
Columns: [country, primary_fuel, capacity_mw, wb_indicator, wb_2020_value]
Index: []


In [12]:
query_missing_capacity = """
SELECT COUNT(*) as missing_capacity
FROM power_plants
WHERE capacity_mw IS NULL
"""
print(pd.read_sql_query(query_missing_capacity, conn))

   missing_capacity
0                 0


In [13]:
# Create a table for total capacity by iso3
conn.execute("DROP TABLE IF EXISTS country_capacity;")

conn.execute("""
CREATE TABLE country_capacity AS
SELECT Country,
       SUM(CASE WHEN primary_fuel IN ('Solar', 'Wind', 'Hydro', 'Geothermal', 'Biomass') 
                THEN capacity_mw ELSE 0 END) AS total_renewable_capacity,
       SUM(capacity_mw) as total_capacity
FROM power_plants
GROUP BY Country;
""")

print("Created 'country_capacity' table with aggregated sums.")

Created 'country_capacity' table with aggregated sums.
